In [63]:
import random
import os
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
import torchvision.models as M
import copy
import logging
import time

In [64]:
if torch.cuda.is_available():
    device = "cuda"
elif torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"

# parameters
n_epoch = 50
batch_size = 1024
lr = 5e-4
input_dim = 2048
output_dim = 1
thresh = 0.5

root = "data/VehicleX/ReID Task/"

In [65]:
def set_random_seed(seed=0):
    # seed setting
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_random_seed()

In [66]:
# get the path and label of the dataset
def path_generator(type):
    paths = []
    labels = []
    # get the root path of the dataset
    type_root = os.path.join(root, type)
    finegrained_labels = os.listdir(type_root)
    # loop each label
    for label in finegrained_labels:
        label_path = os.path.join(type_root, label)
        paths.append(label_path)
        labels.append(int(label.split("_")[0])) 
    return paths, labels

In [67]:
class VehicleDataset(Dataset):
    def __init__(self, type, transform=None):
        paths, labels = path_generator(type)
        self.paths = paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        feature = Image.open(self.paths[idx])
        label = self.labels[idx]
        if self.transform:
            feature = self.transform(feature)
        return feature, label

In [68]:
transforms_src =  T.Compose(
    [
        T.RandomCrop(224),
        T.RandomHorizontalFlip(),
        T.ToTensor(),
        T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]
)

transforms_tar =  T.Compose(
    [
        T.Resize((224,224)),
        T.ToTensor(),
        T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]
)

In [69]:
src_data = VehicleDataset("train", transforms_src)
tar_data = VehicleDataset("gallery", transforms_tar)
tar_test_data = VehicleDataset("query", transforms_tar)

src_loader = DataLoader(src_data, batch_size, shuffle=True)
tar_loader = DataLoader(tar_data, batch_size, shuffle=True)
tar_test_loader = DataLoader(tar_test_data, batch_size, shuffle=True)

# print size of each data
print("src_data size: ", len(src_data))
print("tar_data size: ", len(tar_data))
print("tar_test_data size: ", len(tar_test_data))

print(src_data[0][0].shape)
print(tar_data[0][0].shape)
print(tar_test_data[0][0].shape)

src_data size:  45438
tar_data size:  11579
tar_test_data size:  1678
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])


In [70]:
def show_distribution():
    # check the label distribution of source
    plt.subplot(1, 2, 1)
    src_labels = []
    for i in range(len(src_data)):
        src_labels.append(src_data[i][1])
    plt.hist(src_labels, bins=1362)

    # check the label distribution of query
    plt.subplot(1, 2, 2)
    tar_labels = []
    for i in range(len(tar_test_data)):
        tar_labels.append(tar_test_data[i][1])
    plt.hist(tar_labels, bins=1362)
    plt.show()

# show_distribution()

In [71]:
# baseline model (ResNet34)
class Baseline_ResNet34(nn.Module):
    def __init__(self, output_dim):
        super(Baseline_ResNet34, self).__init__()
        self.resnet34 = M.resnet34(weights=M.ResNet34_Weights.DEFAULT)
        self.feature_extractor = nn.Sequential(*list(self.resnet34.children())[:-1])
        feat_dim = self.resnet34.fc.in_features
        self.clf_fc = nn.Linear(feat_dim, output_dim)

    def forward(self, x):
        x = self.feature_extractor(x)
        x = x.view(x.size(0), -1)
        clf = self.clf_fc(x)
        return x, clf

In [ ]:
class AverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [ ]:
def coral(source, target):
    # my implementation of the original paper, the code is different, but the result is the same
    d = source.data.shape[1]
    ns, nt = source.data.shape[0], target.data.shape[0]
    # source covariance
    # calculating D'D for source and target
    cov_s = source.T @ source
    cov_t = target.T @ target

    # divide D'D by (num-1)
    cov_s = cov_s / (ns - 1)
    cov_t = cov_t / (nt - 1)

    # identity is a row vector of 1s
    identity_s = torch.ones((1, ns), device=source.device)
    identity_t = torch.ones((1, nt), device=target.device)

    # calculate the mean of D per column
    mean_s = identity_s @ source
    mean_t = identity_t @ target

    # calculate the squared mean
    square_mean_s = mean_s.T @ mean_s
    square_mean_t = mean_t.T @ mean_t

    # divide squared mean by (num*(num-1))
    square_mean_s = square_mean_s / (ns * (ns - 1))
    square_mean_t = square_mean_t / (nt * (nt - 1))

    # cov is (1/(num-1))*(D'*D) - (1/(num*(num-1)))*(mean)^T*(mean)
    cov_s = cov_s - square_mean_s
    cov_t = cov_t - square_mean_t

    # cov_s - cov_t
    diff = cov_s - cov_t

    # loss = (1/4)*(1/(dim*dim))*square_norm
    square_norm = torch.sum(torch.multiply(diff, diff))
    loss = square_norm / (4 * d * d)

    return loss

In [ ]:
# train
def train(
        model, 
        src_loader, 
        tar_loader, 
        tar_test_loader, 
        optimizer, 
        args, 
):
    logging.info(f"Train info: lr: {lr}, batch_size: {batch_size}, n_epoch: {n_epoch}, thresh: {thresh}, optimizer: {optimizer}, criterion: {criterion}")
    best_acc = 0.0
    best_model = copy.deepcopy(model.state_dict())
    criterion = nn.CrossEntropyLoss()

    iter_src, iter_tar = iter(src_loader), iter(tar_loader)
    for epoch in range(n_epoch):
        # train
        model.train()
        train_loss_clf = AverageMeter()
        train_loss_transfer = AverageMeter()
        train_loss_total = AverageMeter()

        for _ in range(batch_size):
            data_src, label_src = next(iter_src)
            data_tar, _ = next(iter_tar)
            data_src, label_src = data_src.to(args.device), label_src.to(args.device)
            data_tar = data_tar.to(args.device)

            out_s, clf_s = model(data_src)
            out_t, _ = model(data_tar)
            clf_loss = criterion(clf_s, label_src)
            transfer_loss = coral(out_s, out_t)
            loss = clf_loss + args.transfer_loss_weight * transfer_loss
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss_clf.update(clf_loss.item())
            train_loss_transfer.update(transfer_loss.item())
            train_loss_total.update(loss.item())

        # format in 4 decimal places
        log = f"Epoch: {epoch+1}/{n_epoch}, train_loss_clf: {train_loss_clf.avg:.4f}, 
                train_loss_transfer: {train_loss_transfer.avg:.4f}, 
                train_loss_total: {train_loss_total.avg:.4f},"
        
        # test
        acc, test_loss = test(model, tar_test_loader, args)
        logging.info(f"{log} test_acc: {acc:.4f}, test_loss: {test_loss:.4f}")
        if acc > best_acc:
            best_acc = acc
            best_model = copy.deepcopy(model.state_dict())
    model.load_state_dict(best_model)        
    return model

# test
def test(model, target_test_loader, args):
    model.eval()
    test_loss = AverageMeter()
    correct = 0
    criterion = torch.nn.CrossEntropyLoss()
    len_target_dataset = len(target_test_loader.dataset)
    with torch.no_grad():
        for data, target in target_test_loader:
            data, target = data.to(args.device), target.to(args.device)
            _, clf = model.forward(data)
            loss = criterion(clf, target)
            test_loss.update(loss.item())
            pred = torch.max(clf, 1)[1]
            correct += torch.sum(pred == target)
    acc = 100.0 * correct / len_target_dataset
    return acc, test_loss.avg

In [72]:
model = Baseline_ResNet34(output_dim)
model = model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=lr)

train)